In [ ]:
import pandas as pd
import joblib
from keras.models import Sequential
from keras.layers import *
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

pd.options.mode.chained_assignment = None

# Load data
df = pd.read_csv("house_data.csv")

# Create arrays
X = df[["sq_feet", "num_bedrooms", "num_bathrooms"]]
y = df[["sale_price"]]

# Scaling features
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale training inputs and outputs
X[X.columns] = X_scaler.fit_transform(X[X.columns])
y[y.columns] = y_scaler.fit_transform(y[y.columns])

# Split the data set in a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Neural Network model
model = Sequential()
# input layer
model.add(Dense(50, input_dim=3, activation='relu'))
# 2 hidden layers
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
# output layer
model.add(Dense(1, activation='linear'))
model.compile(
    loss='mean_squared_error',
    optimizer='SGD'
)

# train
model.fit(
    X_train,
    y_train,
    epochs=25,
    batch_size=8,
    shuffle=True,
    verbose=2
)

# Save the scalers to files
joblib.dump(X_scaler, "X_scaler.pkl")
joblib.dump(y_scaler, "y_scaler.pkl")

# Save the trained model
model.save("house_value_model.h5")

print("Model training results:")

# Report mae on the training set 
# scale back to dollars
predictions_train = model.predict(X_train, verbose=0)

mse_train = mean_absolute_error(
    y_scaler.inverse_transform(predictions_train),
    y_scaler.inverse_transform(y_train)
)
print(f" - Training Set Error: {mse_train}")

# Report mae on test set
# scale back to dollars
predictions_test = model.predict(X_test, verbose=0)

mse_test = mean_absolute_error(
    y_scaler.inverse_transform(predictions_test),
    y_scaler.inverse_transform(y_test)
)
print(f" - Test Set Error: {mse_test}")

In [ ]:
# Load trained model
model = load_model('house_value_model.h5')

# Load scalers
X_scaler = joblib.load('X_scaler.pkl')
y_scaler = joblib.load('y_scaler.pkl')

# Define the house that we want to predict
house_1 = [
    500,  # Size in Square Feet
    2,  # Number of Bedrooms
    2,  # Number of Bathrooms
]

homes = [
    house_1
]

# Scale new data
scaled_home_data = X_scaler.transform(homes)

# Make prediction
home_values = model.predict(scaled_home_data)

# prediction from the neural network will be scaled 0 to 1
# unscale
unscaled_home_values = y_scaler.inverse_transform(home_values)

# select first prediction returned
predicted_value = unscaled_home_values[0][0]

print("House details:")
print(f"- {house_1[0]} sq feet")
print(f"- {house_1[1]} bedrooms")
print(f"- {house_1[2]} bathrooms")
print(f"Estimated value: ${predicted_value:,.2f}")